In [1]:
from pathlib import Path
import pprint

# run pip install -e . in the root directory to install this package
from stacbuilder import *

In [2]:
# Collection configuration
catalog_version = "v0.5"
collection_config_path = Path("config-collection.json")

# Input Paths
tiff_input_path = Path("data")
tiffs_glob = '*.tif'

# Output Paths
output_path = Path("results")
test_output_path = output_path / "test" / catalog_version
publish_output_path = output_path / "publish" / catalog_version

# Openeo backend
openeo_backend_url = "https://openeo-dev.vito.be"


In [3]:
# list input files
input_files = list_input_files(
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=None
)
print(f"Path {tiff_input_path}")
print(f"Found {len(input_files)} input files. 5 first files:")
for i in input_files[:5]: print(i) 

Path data
Found 2 input files. 5 first files:
data/Potential_2022.tif
data/Potential_2023.tif


In [4]:
# list meta data
asset_metadata = list_asset_metadata(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1
)
for k in asset_metadata: 
    pprint.pprint(k.to_dict())

{'asset_id': 'Potential_2022',
 'asset_path': PosixPath('data/Potential_2022.tif'),
 'asset_type': 'Potential',
 'bbox_lat_lon': {'east': 11.131896972656252,
                  'epsg': 4326,
                  'north': 52.11609862200532,
                  'south': 52.01500048812244,
                  'west': 10.849857330322266},
 'bbox_projected': {'east': 1239197.1025592776,
                    'epsg': 3857,
                    'north': 6821144.857883772,
                    'south': 6802838.189609473,
                    'west': 1207800.5931911413},
 'collection_id': None,
 'datetime': datetime.datetime(2022, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'end_datetime': datetime.datetime(2022, 12, 31, 23, 59, 59, tzinfo=datetime.timezone.utc),
 'file_size': 12078740,
 'geometry_lat_lon': <POLYGON ((10.8 52, 10.8 52.1, 11.1 52.1, 11.1 52, 10.8 52))>,
 'href': 'data/Potential_2022.tif',
 'item_href': None,
 'item_id': 'Potential_2022',
 'media_type': None,
 'original_href': 'data/Potential

In [5]:
# list items
stac_items, failed_files = list_stac_items(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1
)
print(f"Found {len(stac_items)} STAC items")
if failed_files: print(f"Failed files: {failed_files}")
if len(stac_items) > 0:
    print("First stac item:")
    stac_items[0]

Found 1 STAC items
First stac item:


In [6]:
# build grouped collection
build_grouped_collections(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=True,
)

In [7]:
# validate collection
validate_collection(
    collection_file=test_output_path / "collection.json",
)

In [8]:
# show collection
load_collection(
    collection_file=test_output_path / "collection.json"
)

<Collection id=SENTINEL2_L2A_NDVI_POTENTIAL>

## Legacy
better to use grouped collections

In [9]:
# build collection
build_collection(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=overwrite,
)

NameError: name 'overwrite' is not defined